In [42]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

r1=pd.read_csv('meads_ndvi/ne1_corn_ndvi.csv')
r1=r1.drop('Unnamed: 0',1)
ne1_corn = r1.drop('doy', 1)

r1=pd.read_csv('meads_ndvi/ne1_soy_ndvi.csv')
r1=r1.drop('Unnamed: 0',1)
ne1_soy = r1.drop('doy', 1)

r1=pd.read_csv('meads_ndvi/ne2_corn_ndvi.csv')
r1=r1.drop('Unnamed: 0',1)
ne2_corn = r1.drop('doy', 1)

r1=pd.read_csv('meads_ndvi/ne2_soy_ndvi.csv')
r1=r1.drop('Unnamed: 0',1)
ne2_soy = r1.drop('doy', 1)

r1=pd.read_csv('meads_ndvi/ne3_corn_ndvi.csv')
r1=r1.drop('Unnamed: 0',1)
ne3_corn = r1.drop('doy', 1)

r1=pd.read_csv('meads_ndvi/ne3_soy_ndvi.csv')
r1=r1.drop('Unnamed: 0',1)
ne3_soy = r1.drop('doy', 1)


In [43]:
import os
import pdb

import numpy as np
import pandas as pd
from scipy.optimize import leastsq

def model_fourier(params, agdd, n_harm):
    """
    Fourier model
    :param params:
    :param agdd:
    :param n_harm:
    :return:
    """
    integration_time = len(agdd)
    t = np.arange(1, integration_time + 1)
    result = t*.0 + params[0]
    w = 1

    for i in range(1, n_harm * 4, 4):
        result = result + params[i] * np.cos(2.0 * np.pi * w * t / integration_time + params[i+1]) \
                 + params[i+2]*np.sin(2.0 * np.pi * w * t / integration_time + params[i+3])
        w += 1

    return result


def mismatch_function(params, func_phenology, ndvi, agdd):
    """
    The NDVI/Phenology model mismatch function
    :param params:
    :param func_phenology:
    :param ndvi:
    :param agdd:
    :param years:
    :return:
    """
    # output stores the predictions
    output = []

    oot = ndvi - func_phenology(params, agdd, n_harm=8)

    [output.append(x) for x in oot]

    return np.array(output).squeeze()


def do_fourier(ndvi, gdd, n_harm=8, init_params=None):
    """
    :param ndvi:
    :param gdd:
    :param n_harm:
    :param init_params:
    :return:
    """
    n_params = 1 + n_harm * 4

    if init_params is None:
        init_params = [.25, ] * n_params
        (xsol, mesg) = leastsq(mismatch_function, init_params, args=(model_fourier, ndvi, gdd), maxfev=1000000)
        model_fitted = model_fourier(xsol, gdd, n_harm)

    return model_fitted


def get_PTD(df,gl,gu,sl,su):
    """
    Get phenological transition dates (greenup, senescence)
    :param df:
    :return:
    """
    # Input dataframe has an index comprised of day of year and remaining columns signify NDVI
    # Linearly interpolate dataframe columns to fill in missing values
    df = df.apply(pd.Series.interpolate)

    # Now compute mean of all columns and get the smoothened NDVI
    arr_smooth = do_fourier(df.mean(axis=1), [8.0] * len(df))
    
    """plt.plot(list(range(len(arr_smooth))),arr_smooth)
    plt.xlabel("DOY")
    plt.ylabel("NDVI_smoothened")
    plt.show()"""
    
    # For all other crops and regions, take differential
    # To get doy_green, find the last occurrence of the max differential
    diff_green = np.diff(arr_smooth[:365 + 1])
    """plt.plot(list(range(len(diff_green))),diff_green)
    plt.xlabel("DOY")
    plt.ylabel("NDVI_DIFF")
    plt.show()"""
    #doy_green = np.where(diff_green == diff_green.max())[0][-1]
    #doy_senesc = np.diff(arr_smooth[:365 + 1]).argmin()
    
    green_indices=np.where(diff_green == diff_green[gl:gu+1].max())[0]
    for i in green_indices:
        if i<=gu and i>=gl:
            doy_green = i
            break
    sen_indices=np.where(diff_green == diff_green[sl:su+1].min())[0]
    for i in sen_indices:
        if i<=su and i>=sl:
            doy_senesc = i
            break
    return doy_green, doy_senesc
def g_s_values(icol_start,df,crop):
    green=list()
    sen=list()
    g=0
    s=0
    offset=0#df['doy'][0]-1
    for i in range(icol_start,len(df.columns)):
        if(crop=='corn'):
            gl=140
            gu=176
            sl=213
            su=305
        elif(crop=='soy'):
            gl=156
            gu=196
            sl=227
            su=290
        temp=df.ix[:,i]
        if(temp.first_valid_index()==None):
            green.append(0)
            sen.append(0)
            continue
        temp.iloc[0]=temp.iloc[temp.first_valid_index()]
        g,s=get_PTD(pd.Series.to_frame(temp),gl,gu,sl,su)
        green.append(g+offset)
        sen.append(s+offset)
    return green,sen


In [44]:
green_ne1_corn,sen_ne1_corn=g_s_values(0,ne1_corn,'corn')
green_ne2_corn,sen_ne2_corn=g_s_values(0,ne2_corn,'corn')
green_ne3_corn,sen_ne3_corn=g_s_values(0,ne3_corn,'corn')

green_ne1_soy,sen_ne1_soy=g_s_values(0,ne1_soy,'soy')
green_ne2_soy,sen_ne2_soy=g_s_values(0,ne2_soy,'soy')
green_ne3_soy,sen_ne3_soy=g_s_values(0,ne3_soy,'soy')

C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:122: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [52]:
print("NE1 Corn:\n",green_ne1_corn,'\n',sen_ne1_corn,'\n')
print("NE2 Corn:\n",green_ne2_corn,'\n',sen_ne2_corn,'\n')
print("NE3 Corn:\n",green_ne3_corn,'\n',sen_ne3_corn,'\n')

print("NE1 Soy:\n",green_ne1_soy,'\n',sen_ne1_soy,'\n')
print("NE2 Soy:\n",green_ne2_soy,'\n',sen_ne2_soy,'\n')
print("NE3 Soy:\n",green_ne3_soy,'\n',sen_ne3_soy,'\n')

NE1 Corn:
 [174, 169, 171, 172, 164, 164, 156, 163, 161, 167, 169, 155, 167, 173, 167, 176] 
 [278, 266, 271, 259, 246, 263, 257, 273, 266, 253, 267, 237, 252, 265, 262, 261] 

NE2 Corn:
 [176, 173, 167, 173, 161, 176, 165, 165, 162, 169, 175, 156, 163, 163, 164, 173] 
 [275, 267, 275, 259, 257, 264, 256, 272, 270, 257, 270, 247, 275, 266, 265, 257] 

NE3 Corn:
 [176, 158, 167, 158, 164, 0, 166, 175, 161, 157, 173, 162, 168, 167, 169, 176] 
 [279, 244, 240, 260, 251, 0, 250, 252, 250, 251, 253, 243, 250, 266, 259, 266] 

NE1 Soy:
 [187, 184, 166, 191, 177, 186, 195, 179, 176, 175, 186, 175, 196, 183, 167, 186] 
 [269, 263, 281, 272, 258, 262, 263, 266, 259, 257, 266, 260, 274, 266, 268, 270] 

NE2 Soy:
 [191, 186, 166, 193, 189, 183, 173, 190, 0, 171, 176, 174, 168, 189, 163, 186] 
 [270, 264, 270, 271, 255, 260, 256, 267, 0, 254, 276, 261, 245, 266, 266, 269] 

NE3 Soy:
 [182, 188, 163, 191, 165, 185, 196, 191, 168, 190, 184, 179, 169, 194, 158, 182] 
 [256, 262, 270, 263, 261, 262, 2

In [20]:
print(green)
print(sen)

[174, 169, 171, 172, 164, 164, 156, 163, 161, 167, 169, 155, 167, 173, 167, 176]
[278, 266, 271, 259, 246, 263, 257, 273, 266, 253, 267, 237, 252, 265, 262, 261]


In [ ]:
cleanedList = [x for x in s11['2002_NDVI'].tolist() if str(x) != 'nan']
print(cleanedList)
plt.plot(list(range(len(cleanedList))),cleanedList)
plt.xlabel("DOY")
plt.ylabel("NDVI")
plt.show()

temp=s11.ix[:,'2002_NDVI']
temp.iloc[0]=temp.iloc[temp.first_valid_index()]
print(get_PTD(pd.Series.to_frame(temp)))